### Goal : Generate the configurations for gcc

##### Libraries

In [1]:
import os, time 
import numpy as np
import pandas as pd
from scipy import stats

##### Runtime configs

In [2]:
cols = ["optim", "-floop-interchange", "-fprefetch-loop-arrays", "-ffloat-store", "-fno-asm"]
dico = dict({"optim" : ["-O1", "-O2", "-O0", "-Ofast", "-Og"],
             "-floop-interchange" : ["-floop-interchange",""],
             "-fprefetch-loop-arrays" : ["-fprefetch-loop-arrays",""],
             "-ffloat-store" : ["-ffloat-store",""],
             "-fno-asm" : ["-fno-asm", ""]})

In [3]:
df = pd.DataFrame({})

nb_conf = 1000
nb_conf_final = 80

for d in dico.keys():
    df[d] = np.random.randint(0,len(dico[d]),nb_conf)

df = df.drop_duplicates()[0:nb_conf_final].reset_index().drop(["index"], axis=1)
df

,optim,-floop-interchange,-fprefetch-loop-arrays,-ffloat-store,-fno-asm
0,1,1,1,0,0
1,4,1,1,0,1
2,4,0,1,0,0
3,2,1,0,1,0
4,1,0,0,0,0
...,...,...,...,...,...
75,0,1,0,0,1
76,1,1,0,1,1
77,4,1,1,1,0
78,0,0,0,0,0


In [4]:
df.to_csv('runtime_options.csv')

In [10]:
for j in range(nb_conf_final):
    with open("./scripts/"+str(j)+".sh","w") as f:
        f.write('#!/bin/bash\nnumb="'+str(j))
        f.write('"\nlogfilename="./logs/$numb.log"\ninputlocation="$1"\ninputname="$2"\n')
        f.write("# execute the configuration and measure its time\n")
        vals = df.iloc[j]
        cmd_line = "{ time gcc "
        list_conf = ""
        for i in range(len(vals)):
            if i ==0:
                cmd_line+=dico[cols[i]][vals[i]]+" "
                list_conf+=str(dico[cols[i]][vals[i]])+","
            else:
                cmd_line+=dico[cols[i]][vals[i]]+" "
                val_config = str(dico[cols[i]][vals[i]])
                list_conf+=str(int(val_config == ""))+","*(i!=(len(vals)-1))
        cmd_line+= " -I ./inputs/utilities -I ./inputs/$inputlocation ./inputs/utilities/polybench.c ./inputs/$inputlocation/$inputname.c -DPOLYBENCH_TIME -o ./test; } 2> $logfilename\n"
        f.write(cmd_line)
        f.write("# extract compiled program size \n")
        f.write("size=`ls -lrt ./test | awk '{print $5}'`\n")
        f.write("# analyze log to extract relevant timing information and CPU usage\n")
        f.write("""time=`grep "user" $logfilename | sed 's/user/,/;s/elapsed/,/;s/system/,/' | cut -d "%" -f 1`\n""")
        f.write("# clean\n")
        f.write("""time=`grep "user" $logfilename | sed 's/user/,/;s/elapsed/,/;s/system/,/' | cut -d "%" -f 1`\n""")
        f.write("exec=`./test`\n")
        f.write("eval `rm test`\n")
        f.write('''csvLine="$numb,''')
        f.write(list_conf+'''"\n''')
        f.write('''csvLine="$csvLine,$size,$time,$exec"\n''')
        f.write('''echo $csvLine''')